<a href="https://colab.research.google.com/github/Eran707/MSc-Computational-Neuroscience-Repo/blob/master/Multicompartment_Sim_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Multicompartment simulation V1**

Eran Frank Shorer




## **Import classes and modules** 


In [6]:
import common
import constants
import compartment
import electrodiffusion
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 



## **Set simulation parameters**

## *A) Timing*

In [7]:
dt= 1e-3 # 1ms time steps 
total_t= 1200 #s
run_t = 0 #current simulation timing
t_arr =[0]

## *B) Compartment parameters*

In [11]:
comp_1 = compartment.Compartment("comp_1")
comp_1.set_ion_properties()
comp_2 = compartment.Compartment("comp_2")
comp_2.set_ion_properties()
ed_1_2 = electrodiffusion.Electrodiffusion(comp_1,comp_2)

## **Run Simulation**

In [13]:
while run_t < total_t:

    comp_1.step(dt)
    comp_2.step(dt)
    comp_1_ed_dict = comp_1.get_ed_dict()
    comp_2_ed_dict = comp_2.get_ed_dict()
    ed_conc_changes = ed_1_2.calc_ed(dt,comp_1_ed_dict,comp_2_ed_dict)
    comp_1.ed_update(ed_conc_changes)
    for j in ed_conc_changes:
        ed_conc_changes[j] *= -1
    comp_2.ed_update(ed_conc_changes)
    comp_1.update_volumes()
    comp_1.update_arrays()
    run_t += dt
    t_arr.append(run_t)



TypeError: 'dict_keys' object is not subscriptable